# 二因子變異數分析 - Python實戰：不同廣告與人口變數（地區）如何最佳搭配？

#### 作者：徐子皓(臺灣行銷研究特邀作者)
#### 完整文章介紹鏈接：https://medium.com/p/e310a08d8fe/

## 一、事前檢定

### 1. shapiro常態性檢定

In [5]:
import scipy.stats as st
st.shapiro(alist) 

ShapiroResult(statistic=0.9915865659713745, pvalue=0.6819638013839722)

In [6]:
st.shapiro(blist)

ShapiroResult(statistic=0.9929247498512268, pvalue=0.805463969707489)

In [7]:
st.shapiro(clist)

ShapiroResult(statistic=0.9802003502845764, pvalue=0.07433854043483734)

In [8]:
st.shapiro(dlist)

ShapiroResult(statistic=0.9931642413139343, pvalue=0.7976954579353333)

In [9]:
st.shapiro(elist)

ShapiroResult(statistic=0.988785982131958, pvalue=0.4260871410369873)

In [10]:
st.shapiro(flist)

ShapiroResult(statistic=0.9914966225624084, pvalue=0.718219518661499)

### 2. levene同質性檢定

In [11]:
st.levene(alist, blist, clist, center='mean')

LeveneResult(statistic=0.6891243668404422, pvalue=0.5026819827621525)

In [12]:
st.levene(dlist, elist, flist, center='mean')

LeveneResult(statistic=0.25909843233185653, pvalue=0.7718920508101443)

## 二、多因子變異數檢定

### 1. 建立模型並導入資料

In [14]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('消費金額 ~ C(廣告) + C(地區) + C(廣告):C(地區)', data=data).fit()

### 2. 產出分析結果

In [15]:
Results = sm.stats.anova_lm(model)
Results

,df,sum_sq,mean_sq,F,PR(>F)
C(廣告),2.0,173.124185,86.562092,2.367086,0.095249
C(地區),2.0,10.030991,5.015496,0.137151,0.871885
C(廣告):C(地區),4.0,228.995049,57.248762,1.565497,0.183004
Residual,351.0,12835.738552,36.569056,NaN,NaN


## 三、財務交叉比較表

### 1. 取出要素名稱

In [16]:
var1 = data['廣告'].unique().tolist()
var2 = data['地區'].unique().tolist()

In [17]:
var1

['廣告1', '廣告2', '廣告3']

In [18]:
var2

['南部', '中部', '北部']

### 2. 將因素交叉組合，計算各自平均消費金額

In [19]:
ad1_list = []
ad2_list = []
ad3_list = []
for i in var1:
    for ii in var2:
        if i == '廣告1':
            ad1_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())
        elif i == '廣告2':
            ad2_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())
        elif i == '廣告3':
            ad3_list.append(data[data ['廣告'] == i][data ['地區'] == ii]['消費金額'].mean())

### 3. 查看交叉結果

In [20]:
ad1_list

[17.26818181818182, 17.9854347826087, 16.074146341463415]

In [21]:
ad2_list

[15.513399999999999, 14.253636363636364, 16.89324324324324]

In [22]:
ad3_list

[14.786896551724137, 16.198809523809523, 15.930816326530612]

### 4. 製作交叉比較表

In [23]:
cross_table = pd.DataFrame((ad1_list, ad2_list, ad3_list),             columns = var2, index = var1)
cross_table

,南部,中部,北部
廣告1,17.268182,17.985435,16.074146
廣告2,15.513400,14.253636,16.893243
廣告3,14.786897,16.198810,15.930816


## 四、資料視覺化

### 1. 基本資料輸入

In [24]:
import plotly.offline as py
import plotly.graph_objects as go
fig = go.Figure()
# 廣告1分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[0,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="royalblue"),    
            line=dict(color='royalblue', width=2),
            name = '廣告1'
            ))
# 廣告2分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[1,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="firebrick"),    
            line=dict(color='firebrick', width=2),
            name = '廣告2'
            ))
# 廣告3分布圖
fig.add_trace(go.Scatter(
            x= var2,
            y= cross_table.iloc[2,:].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="green"),    
            line=dict(color='green', width=2),
            name = '廣告3'
            ))

### 2. 設定佈景主題

In [25]:
fig.update_layout(
    title={
        'text': "<b>廣告效益分析</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',},
    yaxis_title='平均消費金額',
    xaxis={
        'title': '地區',
        'tickmode': 'linear'
        },
    width=1800,
    height=960,
    boxmode='group',
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="lightslategrey"
    )
    )

### 3. 將產出圖片另存

In [26]:
# 另存html檔
py.plot(fig, filename='多因子變異數分析', auto_open=True)
# 另存png檔
fig.write_image("C:/Users/user/Desktop/多因子變異數分析.png")

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\user\AppData\Roaming\Python\Python36\site-packages\pywin32_system32
    C:\Users\user\Anaconda3\envs\win
    C:\Users\user\Anaconda3\envs\win\Library\mingw-w64\bin
    C:\Users\user\Anaconda3\envs\win\Library\usr\bin
    C:\Users\user\Anaconda3\envs\win\Library\bin
    C:\Users\user\Anaconda3\envs\win\Scripts
    C:\Users\user\Anaconda3\envs\win\bin
    C:\Users\user\Anaconda3\condabin
    C:\WINDOWS\system32
    C:\WINDOWS
    C:\WINDOWS\System32\Wbem
    C:\WINDOWS\System32\WindowsPowerShell\v1.0
    C:\WINDOWS\System32\OpenSSH
    C:\Users\user\AppData\Local\Microsoft\WindowsApps
    .
    C:\Users\user\AppData\Local\Programs\Microsoft VS Code\bin
    C:\Users\user\Anaconda3\envs\win\lib\site-packages\numpy\.libs
    C:\Users\user\Anaconda3\envs\win\lib\site-packages\scipy\.libs

If you have